### Conv on aminos only

In [ ]:
my_learner.load("Complex_aminos_1")

In [ ]:
batch_size = 32
#secret_size = 401
n_hid = 20
drop_prob = 0
ks1 = 1
ks2 = 3
my_data_bunch = DataBunch.create(train_ds, valid_ds, bs=batch_size)
criterion = nn.CrossEntropyLoss()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        #Conv1
        self.conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 10)
        self.max_1 = nn.MaxPool1d(kernel_size = 3)
        self.drop_1 = nn.Dropout(p = drop_prob)
        self.selu_1 = nn.SELU()
        
        self.linear_4 = nn.Linear(in_features = 153, out_features = 1)
        self.selu_4 = nn.SELU()
        
        self.linear_last = nn.Linear(in_features = n_hid, out_features = 2)
        self.logsoft_last = nn.LogSoftmax()

    def forward(self, x):
        bs0 = x.shape[0]
        x = x[:, :, 0:20]
        print("\nInput", x.shape)
        x = x.transpose(1, 2)
        print("Transpose", x.shape)
        
        x = self.selu_1(self.drop_1(self.max_1(self.conv_1(x))))
        print("Conv", x.shape)
        
        x = self.linear_4(x)
        x = self.selu_4(x)
        print("Lin4", x.shape)
        
        x = x.reshape(bs0, n_hid)
        print("Reshape", x.shape)
        x = self.linear_last(x)
        print("Last", x.shape)
        x = self.logsoft_last(x)
        print("Logged", x.shape)
        return x

#print(net)  
#print( 'trainable params: {}'.format(count_model_params(net)) )
net = Model().cuda()

my_learner = Learner(my_data_bunch,
                     net,
                     opt_func=torch.optim.Adam,
                     loss_func=criterion, metrics=accuracy)
#my_learner.fit_one_cycle(3, 1e-03)


In [ ]:
my_learner.fit_one_cycle(3, 1e-3, wd = 0.01) #Ca 8 epochs with dropout = 0.3, wd = 0.01

- Correct: 0.784
- AUC 0.87
- MCC 0.587
- AVP 0.863

- Tn, fp, fn, tp
- [[196  16]
-  [ 73 127]]
- TPR: 0.635
- TNR: 0.925

###  Improved aminos only

In [ ]:
my_learner.load("Complex_aminos_1")

In [ ]:
batch_size = 32
#secret_size = 401
n_hid = 20
drop_prob = 0.3
ks1 = 1
ks2 = 3
my_data_bunch = DataBunch.create(train_ds, valid_ds, bs=batch_size)
criterion = nn.CrossEntropyLoss()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        #Conv1
        self.conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 10)
        self.max_1 = nn.MaxPool1d(kernel_size = 3)
        self.drop_1 = nn.Dropout(p = drop_prob)
        self.selu_1 = nn.SELU()
    
        self.linear_4 = nn.Linear(in_features = 153, out_features = 1)
        self.selu_4 = nn.SELU()
        
        self.linear_last = nn.Linear(in_features = n_hid, out_features = 2)
        self.logsoft_last = nn.LogSoftmax()

    def forward(self, x):
        bs0 = x.shape[0]
        x = x[:, :, :20]
        print("\nInput", x.shape)
        x = x.transpose(1, 2)
        print("Transpose", x.shape)
        
        x = self.selu_1(self.drop_1(self.max_1(self.conv_1(x))))
        print("Conv", x.shape)
        
        x = self.linear_4(x)
        x = self.selu_4(x)
        print("Lin4", x.shape)
        
        x = x.reshape(bs0, n_hid)
        print("Reshape", x.shape)
        x = self.linear_last(x)
        print("Last", x.shape)
        x = self.logsoft_last(x)
        print("Logged", x.shape)
        return x

#print(net)  
#print( 'trainable params: {}'.format(count_model_params(net)) )
net = Model().cuda()

my_learner = Learner(my_data_bunch,
                     net,
                     opt_func=torch.optim.Adam,
                     loss_func=criterion, metrics=accuracy)
#my_learner.fit_one_cycle(3, 1e-03)


In [1]:
Correct: 0.799
AUC 0.874
MCC 0.607
AVP 0.866

Tn, fp, fn, tp
[[191  21]
 [ 62 138]]
TPR: 0.69
TNR: 0.901

SyntaxError: invalid syntax (<ipython-input-1-1e534aa11f4a>, line 2)

### Single layer work on both parts

In [ ]:
my_learner.load("Complex_aminos_6")

In [4]:
## batch_size = 32
#secret_size = 401
n_hid = 20
drop_prob = 0
ks1 = 1
ks2 = 3
my_data_bunch = DataBunch.create(train_ds, valid_ds, bs=batch_size)
criterion = nn.CrossEntropyLoss()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        #Pad
        self.m_pad = nn.ConstantPad1d(160, 0)
        self.t_pad = nn.ConstantPad1d(118, 0)
        
        #Conv1
        self.m_conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_1 = nn.BatchNorm1d(20)
        self.m_max_1 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_1 = nn.Dropout(p = drop_prob)
        self.m_ReLU_1 = nn.ReLU()

        self.t_conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_1 = nn.BatchNorm1d(20)
        self.t_max_1 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_1 = nn.Dropout(p = drop_prob)
        self.t_ReLU_1 = nn.ReLU()
        self.t_norm_1 = nn.BatchNorm1d(20)
        
        #Conv2
        self.m_conv_2 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_2 = nn.BatchNorm1d(20)
        self.m_max_2 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_2 = nn.Dropout(p = drop_prob)
        self.m_ReLU_2 = nn.ReLU()

        self.t_conv_2 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_2 = nn.BatchNorm1d(20)
        self.t_max_2 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_2 = nn.Dropout(p = drop_prob)
        self.t_ReLU_2 = nn.ReLU()
        
        
        #Conv3
        self.m_conv_3 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_3 = nn.BatchNorm1d(20)
        self.m_max_3 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_3 = nn.Dropout(p = drop_prob)
        self.m_ReLU_3 = nn.ReLU()

        self.t_conv_3 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_3 = nn.BatchNorm1d(20)
        self.t_max_3 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_3 = nn.Dropout(p = drop_prob)
        self.t_ReLU_3 = nn.ReLU()
        
        #Conv4
        self.m_conv_4 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_4 = nn.BatchNorm1d(20)
        self.m_max_4 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_4 = nn.Dropout(p = drop_prob)
        self.m_ReLU_4 = nn.ReLU()

        self.t_conv_4 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_4 = nn.BatchNorm1d(20)
        self.t_max_4 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_4 = nn.Dropout(p = drop_prob)
        self.t_ReLU_4 = nn.ReLU()
        
        #Conv5
        self.m_conv_5 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_5 = nn.BatchNorm1d(20)
        self.m_max_5 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_5 = nn.Dropout(p = drop_prob)
        self.m_ReLU_5 = nn.ReLU()

        self.t_conv_5 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_5 = nn.BatchNorm1d(20)
        self.t_max_5 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_5 = nn.Dropout(p = drop_prob)
        self.t_ReLU_5 = nn.ReLU()
        
        #Random projection module
        
        
        #Linear
        self.a_linear = nn.Linear(in_features = 320, out_features = 2)
        self.a_selu = nn.SELU()
        
        #self.linear_last = nn.Linear(in_features = n_hid, out_features = 2)
        #self.logsoft_last = nn.LogSoftmax()

    def forward(self, x):
        bs0 = x.shape[0]
        x = x[:, :, :20]
        
        print("\nInput", x.shape)
        x = x.transpose(1, 2)
        print("Transpose", x.shape)
        
        mhcp = x[:, :, 0:192]
        tcr = x[:, :, 192:]
        print("Shapes", mhcp.shape, tcr.shape)
        
        mhcp0 = self.m_pad(mhcp)
        tcr0 = self.t_pad(tcr)
        print("Padded", mhcp0.shape, tcr0.shape)
        
        mhcp = self.m_max_1(self.m_drop_1(self.m_ReLU_1(self.m_norm_1(self.m_conv_1(mhcp0)))))
        tcr = self.m_ReLU_1(self.m_drop_1(self.m_max_1(self.m_norm_1(self.m_conv_1(tcr0)))))
        print("Conv1", mhcp.shape, tcr.shape)

        mhcp = self.m_max_2(self.m_drop_2(self.m_ReLU_2(self.m_norm_2(self.m_conv_2(mhcp)))))
        tcr = self.m_ReLU_2(self.m_drop_2(self.m_max_2(self.m_norm_2(self.m_conv_2(tcr)))))
        print("Conv2", mhcp.shape, tcr.shape)

        mhcp = self.m_max_3(self.m_drop_3(self.m_ReLU_3(self.m_norm_3(self.m_conv_3(mhcp)))))
        tcr = self.m_ReLU_3(self.m_drop_3(self.m_max_3(self.m_norm_3(self.m_conv_3(tcr)))))
        print("Conv3", mhcp.shape, tcr.shape)

        mhcp = self.m_max_4(self.m_drop_4(self.m_ReLU_4(self.m_norm_4(self.m_conv_4(mhcp)))))
        tcr = self.m_ReLU_4(self.m_drop_4(self.m_max_4(self.m_norm_4(self.m_conv_4(tcr)))))
        print("Conv4", mhcp.shape, tcr.shape)

        mhcp = self.m_max_5(self.m_drop_5(self.m_ReLU_5(self.m_norm_5(self.m_conv_5(mhcp)))))
        tcr = self.m_ReLU_5(self.m_drop_5(self.m_max_5(self.m_norm_5(self.m_conv_5(tcr)))))
        print("Conv5", mhcp.shape, tcr.shape)

        
        mhcp = mhcp.view(bs0, 320)
        tcr = tcr.view(bs0, 320)
        print("Views", mhcp.shape, tcr.shape)
    
        #Random projection module
        #transformer = random_projection.SparseRandomProjection(n_components=32, random_state=1, eps = 0.001)
        #mhcp = torch.from_numpy(transformer.fit_transform(mhcp)).float().cuda()
        #tcr = torch.from_numpy(transformer.fit_transform(tcr)).float().cuda()
        
        print("Random transform", mhcp.shape, tcr.shape)
        print("Types", type(mhcp), type(tcr))
        print(mhcp)
        
        allparts = mhcp * tcr
        print("Allparts", allparts.shape)
        
        #Linear module
        allparts = self.a_linear(allparts)
        print("Last", allparts.shape)
        
        x = allparts
        return x

#print(net)  
#print( 'trainable params: {}'.format(count_model_params(net)) )
net = Model().cuda()

my_learner = Learner(my_data_bunch,
                     net,
                     opt_func=torch.optim.Adam,
                     loss_func=criterion, metrics=accuracy)
my_learner.fit_one_cycle(1, 1e-02, wd = 0.1)


NameError: name 'DataBunch' is not defined

In [1]:
my_learner.fit_one_cycle(1, 1e-02, wd = 0.1)
my_learner.fit_one_cycle(3, 1e-2, wd = 0.01)

NameError: name 'my_learner' is not defined

In [ ]:
Correct: 0.915
AUC 0.962
MCC 0.83
AVP 0.935

Tn, fp, fn, tp
[[196  16]
 [ 19 181]]
TPR: 0.905
TNR: 0.925

### SELU win, dropprob 0.1

In [ ]:
my_learner.load("SELUwin1")

In [ ]:
batch_size = 32
#secret_size = 401
n_hid = 20
drop_prob = 0.1
ks1 = 1
ks2 = 3
my_data_bunch = DataBunch.create(train_ds, valid_ds, bs=batch_size)
criterion = nn.CrossEntropyLoss()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        #Pad
        self.m_pad = nn.ConstantPad1d(160, 0)
        self.t_pad = nn.ConstantPad1d(118, 0)
        
        #Conv1
        self.m_conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_1 = nn.BatchNorm1d(20)
        self.m_max_1 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_1 = nn.Dropout(p = drop_prob)
        self.m_ReLU_1 = nn.ReLU()

        self.t_conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_1 = nn.BatchNorm1d(20)
        self.t_max_1 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_1 = nn.Dropout(p = drop_prob)
        self.t_ReLU_1 = nn.ReLU()
        self.t_norm_1 = nn.BatchNorm1d(20)

        #Conv2
        self.m_conv_2 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_2 = nn.BatchNorm1d(20)
        self.m_max_2 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_2 = nn.Dropout(p = drop_prob)
        self.m_ReLU_2 = nn.ReLU()

        self.t_conv_2 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_2 = nn.BatchNorm1d(20)
        self.t_max_2 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_2 = nn.Dropout(p = drop_prob)
        self.t_ReLU_2 = nn.ReLU()


        #Conv3
        self.m_conv_3 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_3 = nn.BatchNorm1d(20)
        self.m_max_3 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_3 = nn.Dropout(p = drop_prob)
        self.m_ReLU_3 = nn.ReLU()

        self.t_conv_3 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_3 = nn.BatchNorm1d(20)
        self.t_max_3 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_3 = nn.Dropout(p = drop_prob)
        self.t_ReLU_3 = nn.ReLU()

        #Conv4
        self.m_conv_4 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_4 = nn.BatchNorm1d(20)
        self.m_max_4 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_4 = nn.Dropout(p = drop_prob)
        self.m_ReLU_4 = nn.ReLU()

        self.t_conv_4 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_4 = nn.BatchNorm1d(20)
        self.t_max_4 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_4 = nn.Dropout(p = drop_prob)
        self.t_ReLU_4 = nn.ReLU()

        #Conv5
        self.m_conv_5 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_5 = nn.BatchNorm1d(20)
        self.m_max_5 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_5 = nn.Dropout(p = drop_prob)
        self.m_ReLU_5 = nn.ReLU()

        self.t_conv_5 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_5 = nn.BatchNorm1d(20)
        self.t_max_5 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_5 = nn.Dropout(p = drop_prob)
        self.t_ReLU_5 = nn.ReLU()
        
        #Random projection module
        self.mrp_Linear = nn.Linear(in_features = 640, out_features = 640)
        self.mrp_norm = nn.BatchNorm1d(640)
        self.mrp_ReLU = nn.ReLU()
        
        self.trp_Linear = nn.Linear(in_features = 640, out_features = 640)
        self.trp_norm = nn.BatchNorm1d(640)
        self.trp_ReLU = nn.ReLU()
        
        
        print("m1", m1.shape)
        
        #Linear last layer
        self.a_linear = nn.Linear(in_features = 640, out_features = 2)
        self.a_sigmoid = nn.SELU()

        
        #self.linear_last = nn.Linear(in_features = n_hid, out_features = 2)
        #self.logsoft_last = nn.LogSoftmax()

    def forward(self, x):
        bs0 = x.shape[0]
        x = x[:, :, :20]
        
        print("\nInput", x.shape)
        x = x.transpose(1, 2)
        print("Transpose", x.shape)
        
        mhcp = x[:, :, 0:192]
        tcr = x[:, :, 192:]
        print("Shapes", mhcp.shape, tcr.shape)
        
        mhcp0 = self.m_pad(mhcp)
        tcr0 = self.t_pad(tcr)
        print("Padded", mhcp0.shape, tcr0.shape)
        
        mhcp = self.m_max_1(self.m_drop_1(self.m_ReLU_1(self.m_norm_1(self.m_conv_1(mhcp0)))))
        tcr = self.m_max_1(self.m_drop_1(self.m_ReLU_1(self.m_norm_1(self.m_conv_1(tcr0)))))
        print("Conv1", mhcp.shape, tcr.shape)

        mhcp = self.m_max_2(self.m_drop_2(self.m_ReLU_2(self.m_norm_2(self.m_conv_2(mhcp)))))
        tcr = self.m_max_2(self.m_drop_2(self.m_ReLU_2(self.m_norm_2(self.m_conv_2(tcr)))))
        print("Conv2", mhcp.shape, tcr.shape)

        mhcp = self.m_max_3(self.m_drop_3(self.m_ReLU_3(self.m_norm_3(self.m_conv_3(mhcp)))))
        tcr = self.m_max_3(self.m_drop_3(self.m_ReLU_3(self.m_norm_3(self.m_conv_3(tcr)))))
        print("Conv3", mhcp.shape, tcr.shape)

        mhcp = self.m_max_4(self.m_drop_4(self.m_ReLU_4(self.m_norm_4(self.m_conv_4(mhcp)))))
        tcr = self.m_max_4(self.m_drop_4(self.m_ReLU_4(self.m_norm_4(self.m_conv_4(tcr)))))
        print("Conv4", mhcp.shape, tcr.shape)


        mhcp = mhcp.view(bs0, 640)
        tcr = tcr.view(bs0, 640)
        print("Views", mhcp.shape, tcr.shape)
        
        mhcp = self.mrp_ReLU(self.mrp_norm(self.mrp_Linear(mhcp)))
        tcr = self.trp_ReLU(self.trp_norm(self.trp_Linear(tcr)))
        print("Pre-rp", mhcp.shape, tcr.shape)
        
        #Random projection -> 640
        #mhcp = mhcp * m1[0:bs0]
        #tcr = tcr * m2[0:bs0]
        print("RPed", mhcp.shape, tcr.shape)
        
        allparts = mhcp * tcr
        print("Allparts", allparts.shape)
        
        #Linear module
        allparts = self.a_linear(allparts)
        allparts = self.a_sigmoid(allparts)
        print("Last", allparts.shape)
        
        x = allparts
        return x

#print(net)  
#print( 'trainable params: {}'.format(count_model_params(net)) )
net = Model().cuda()

my_learner = Learner(my_data_bunch,
                     net,
                     opt_func=torch.optim.Adam,
                     loss_func=criterion, metrics=accuracy)
#my_learner.fit_one_cycle(1, 1e-02, wd = 0.01)


In [ ]:
my_learner.fit_one_cycle(5, 1e-2, wd = 0.01)

### my_learner.load("Relu0871")

- Gaussian distribution
- ReLU at end
- Drop_prob 0, wd 0
- LR 1e-02, ca 10 epochs

In [ ]:
batch_size = 32
#secret_size = 401
n_hid = 20
drop_prob = 0
ks1 = 1
ks2 = 3
my_data_bunch = DataBunch.create(train_ds, valid_ds, bs=batch_size)
criterion = nn.CrossEntropyLoss()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        #Pad
        self.m_pad = nn.ConstantPad1d(160, 0)
        self.t_pad = nn.ConstantPad1d(118, 0)
        
        #Conv1
        self.m_conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_1 = nn.BatchNorm1d(20)
        self.m_max_1 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_1 = nn.Dropout(p = drop_prob)
        self.m_ReLU_1 = nn.ReLU()

        self.t_conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_1 = nn.BatchNorm1d(20)
        self.t_max_1 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_1 = nn.Dropout(p = drop_prob)
        self.t_ReLU_1 = nn.ReLU()
        self.t_norm_1 = nn.BatchNorm1d(20)

        #Conv2
        self.m_conv_2 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_2 = nn.BatchNorm1d(20)
        self.m_max_2 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_2 = nn.Dropout(p = drop_prob)
        self.m_ReLU_2 = nn.ReLU()

        self.t_conv_2 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_2 = nn.BatchNorm1d(20)
        self.t_max_2 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_2 = nn.Dropout(p = drop_prob)
        self.t_ReLU_2 = nn.ReLU()


        #Conv3
        self.m_conv_3 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_3 = nn.BatchNorm1d(20)
        self.m_max_3 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_3 = nn.Dropout(p = drop_prob)
        self.m_ReLU_3 = nn.ReLU()

        self.t_conv_3 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_3 = nn.BatchNorm1d(20)
        self.t_max_3 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_3 = nn.Dropout(p = drop_prob)
        self.t_ReLU_3 = nn.ReLU()

        #Conv4
        self.m_conv_4 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_4 = nn.BatchNorm1d(20)
        self.m_max_4 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_4 = nn.Dropout(p = drop_prob)
        self.m_ReLU_4 = nn.ReLU()

        self.t_conv_4 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_4 = nn.BatchNorm1d(20)
        self.t_max_4 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_4 = nn.Dropout(p = drop_prob)
        self.t_ReLU_4 = nn.ReLU()

        #Conv5
        self.m_conv_5 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.m_norm_5 = nn.BatchNorm1d(20)
        self.m_max_5 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.m_drop_5 = nn.Dropout(p = drop_prob)
        self.m_ReLU_5 = nn.ReLU()

        self.t_conv_5 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_5 = nn.BatchNorm1d(20)
        self.t_max_5 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_5 = nn.Dropout(p = drop_prob)
        self.t_ReLU_5 = nn.ReLU()

        self.t_conv_5 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = 11, padding = 5)
        self.t_norm_5 = nn.BatchNorm1d(20)
        self.t_max_5 = nn.AvgPool1d(kernel_size = 2, stride = 2)
        self.t_drop_5 = nn.Dropout(p = drop_prob)
        self.t_ReLU_5 = nn.SELU()
        
        #Random projection module
        self.mrp_Linear = nn.Linear(in_features = 640, out_features = 640)
        self.mrp_norm = nn.BatchNorm1d(640)
        self.mrp_ReLU = nn.ReLU()
        
        self.trp_Linear = nn.Linear(in_features = 640, out_features = 640)
        self.trp_norm = nn.BatchNorm1d(640)
        self.trp_ReLU = nn.Sigmoid()
        
        
        print("m1", m1.shape)
        
        #Linear last layer
        self.a_linear = nn.Linear(in_features = 640, out_features = 2)
        self.a_sigmoid = nn.Sigmoid()

        
        #self.linear_last = nn.Linear(in_features = n_hid, out_features = 2)
        #self.logsoft_last = nn.LogSoftmax()

    def forward(self, x):
        bs0 = x.shape[0]
        x = x[:, :, :20]
        
        print("\nInput", x.shape)
        x = x.transpose(1, 2)
        print("Transpose", x.shape)
        
        mhcp = x[:, :, 0:192]
        tcr = x[:, :, 192:]
        print("Shapes", mhcp.shape, tcr.shape)
        
        mhcp0 = self.m_pad(mhcp)
        tcr0 = self.t_pad(tcr)
        print("Padded", mhcp0.shape, tcr0.shape)
        
        mhcp = self.m_max_1(self.m_drop_1(self.m_ReLU_1(self.m_norm_1(self.m_conv_1(mhcp0)))))
        tcr = self.t_max_1(self.t_drop_1(self.t_ReLU_1(self.t_norm_1(self.t_conv_1(tcr0)))))
        print("Conv1", mhcp.shape, tcr.shape)

        mhcp = self.m_max_2(self.m_drop_2(self.m_ReLU_2(self.m_norm_2(self.m_conv_2(mhcp)))))
        tcr = self.t_max_2(self.t_drop_2(self.t_ReLU_2(self.t_norm_2(self.t_conv_2(tcr)))))
        print("Conv2", mhcp.shape, tcr.shape)

        mhcp = self.m_max_3(self.m_drop_3(self.m_ReLU_3(self.m_norm_3(self.m_conv_3(mhcp)))))
        tcr = self.t_max_3(self.t_drop_3(self.t_ReLU_3(self.t_norm_3(self.t_conv_3(tcr)))))
        print("Conv3", mhcp.shape, tcr.shape)

        mhcp = self.m_max_4(self.m_drop_4(self.m_ReLU_4(self.m_norm_4(self.m_conv_4(mhcp)))))
        tcr = self.t_max_4(self.t_drop_4(self.t_ReLU_4(self.t_norm_4(self.t_conv_4(tcr)))))
        print("Conv4", mhcp.shape, tcr.shape)

        mhcp = mhcp.view(bs0, 640)
        tcr = tcr.view(bs0, 640)
        print("Views", mhcp.shape, tcr.shape)
        
        mhcp = self.mrp_ReLU(self.mrp_norm(self.mrp_Linear(mhcp)))
        tcr = self.trp_ReLU(self.trp_norm(self.trp_Linear(tcr)))
        print("Pre-rp", mhcp.shape, tcr.shape)
        
        #Random projection -> 640
        mhcp = mhcp * m1[0:bs0]
        tcr = tcr * m2[0:bs0]
        print("RPed", mhcp.shape, tcr.shape)
        
        allparts = mhcp * tcr
        print("Allparts", allparts.shape)
        
        #Linear module
        allparts = self.a_linear(allparts)
        allparts = self.a_sigmoid(allparts)
        print("Last", allparts.shape)
        
        x = allparts
        return x

#print(net)  
#print( 'trainable params: {}'.format(count_model_params(net)) )
net = Model().cuda()

my_learner = Learner(my_data_bunch,
                     net,
                     opt_func=torch.optim.Adam,
                     loss_func=criterion, metrics=accuracy)
#my_learner.fit_one_cycle(1, 1e-02, wd = 0.01)


In [ ]:
Correct: 0.871
AUC 0.942
MCC 0.746
AVP 0.931

Tn, fp, fn, tp
[[196  16]
 [ 37 163]]
TPR: 0.815
TNR: 0.925

In [ ]:
#test set
Correct: 0.9
AUC 1.0
MCC 0.815
AVP 1.0

Tn, fp, fn, tp
[[108   0]
 [ 21  81]]
TPR: 0.794
TNR: 1.0

### Resnet idea 086

In [ ]:
my_learner.load("Alldata_086_Resnetidea")

In [ ]:
batch_size = 32
#secret_size = 401
n_hid = 56
drop_prob = 0.1
ks1 = 11
pad1 = int((ks1) / 2)

ks2 = 3
epochs = 0
ps = True
my_data_bunch = DataBunch.create(train_ds, valid_ds, bs=batch_size)
#criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        #Pad
        self.m_pad = nn.ConstantPad1d((160), 0)
        self.t_pad = nn.ConstantPad1d((118), 0)
        #self.p_pad = nn.ConstantPad1d((112, 113), 0)
        
        #Conv1
        self.m_conv_1 = nn.Conv1d(in_channels = 56, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.m_norm_1 = nn.BatchNorm1d(n_hid)
        self.m_max_1 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_1 = nn.Dropout(p = drop_prob)
        self.m_ReLU_1 = nn.ReLU()

        self.t_conv_1 = nn.Conv1d(in_channels = 56, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.t_norm_1 = nn.BatchNorm1d(n_hid)
        self.t_max_1 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_1 = nn.Dropout(p = drop_prob)
        self.t_ReLU_1 = nn.ReLU()

        #self.p_conv_1 = nn.Conv1d(in_channels = 20, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        #self.p_norm_1 = nn.BatchNorm1d(n_hid)
        #self.p_max_1 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        #self.p_drop_1 = nn.Dropout(p = drop_prob)
        #self.p_ReLU_1 = nn.ReLU()

        #Conv2
        self.m_conv_2 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.m_norm_2 = nn.BatchNorm1d(n_hid)
        self.m_max_2 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_2 = nn.Dropout(p = drop_prob)
        self.m_ReLU_2 = nn.ReLU()

        self.t_conv_2 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.t_norm_2 = nn.BatchNorm1d(n_hid)
        self.t_max_2 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_2 = nn.Dropout(p = drop_prob)
        self.t_ReLU_2 = nn.ReLU()

        #self.p_conv_2 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        #self.p_norm_2 = nn.BatchNorm1d(n_hid)
        #self.p_max_2 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        #self.p_drop_2 = nn.Dropout(p = drop_prob)
        #self.p_ReLU_2 = nn.ReLU()


        #Conv3
        self.m_conv_3 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.m_norm_3 = nn.BatchNorm1d(n_hid)
        self.m_max_3 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_3 = nn.Dropout(p = drop_prob)
        self.m_ReLU_3 = nn.ReLU()

        self.t_conv_3 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.t_norm_3 = nn.BatchNorm1d(n_hid)
        self.t_max_3 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_3 = nn.Dropout(p = drop_prob)
        self.t_ReLU_3 = nn.ReLU()

        #self.p_conv_3 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        #self.p_norm_3 = nn.BatchNorm1d(n_hid)
        #self.p_max_3 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        #self.p_drop_3 = nn.Dropout(p = drop_prob)
        #self.p_ReLU_3 = nn.ReLU()

        #Conv4
        self.m_conv_4 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.m_norm_4 = nn.BatchNorm1d(n_hid)
        self.m_max_4 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.m_drop_4 = nn.Dropout(p = drop_prob)
        self.m_ReLU_4 = nn.ReLU()

        self.t_conv_4 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        self.t_norm_4 = nn.BatchNorm1d(n_hid)
        self.t_max_4 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        self.t_drop_4 = nn.Dropout(p = drop_prob)
        self.t_ReLU_4 = nn.ReLU()

        #self.p_conv_4 = nn.Conv1d(in_channels = n_hid, out_channels = n_hid, kernel_size = ks1, padding = pad1)
        #self.p_norm_4 = nn.BatchNorm1d(n_hid)
        #self.p_max_4 = nn.MaxPool1d(kernel_size = 2, stride = 2)
        #self.p_drop_4 = nn.Dropout(p = drop_prob)
        #self.p_ReLU_4 = nn.ReLU()

        #Random projection module
        self.mrp_Linear = nn.Linear(in_features = (32*n_hid), out_features = 32*n_hid)
        self.mrp_norm = nn.BatchNorm1d(32*n_hid)
        self.mrp_ReLU = nn.ReLU()
        
        self.trp_Linear = nn.Linear(in_features = 32*n_hid, out_features = 32*n_hid)
        self.trp_norm = nn.BatchNorm1d(32*n_hid)
        self.trp_ReLU = nn.ReLU()
        
        #self.prp_Linear = nn.Linear(in_features = 32*n_hid, out_features = 32*n_hid)
        #self.prp_norm = nn.BatchNorm1d(32*n_hid)
        #self.prp_ReLU = nn.ReLU()
        
        #Linear last layer
        self.a_linear = nn.Linear(in_features = 32*n_hid, out_features = 2)
        self.a_sigmoid = nn.Sigmoid()

    def forward(self, x):
        global ps
        bs0 = x.shape[0]
        #x = x[:, :, :20]
        x = x[:, :, :]
        
        if ps: print("\nInput", x.shape)
        x = x.transpose(1, 2)
        if ps: print("Transpose", x.shape)
        
        mhcp = x[:, :, 0:192]
        tcr = x[:, :, 192:]
        pep = x[:, :, 181:192]
        pep0 = pep
        #tcr = torch.cat((tcr, pep), dim = 2)
        if ps: print("Shapes", mhcp.shape, tcr.shape, pep.shape)

        mhcp0 = self.m_pad(mhcp)
        tcr0 = self.t_pad(tcr)
        #pep0 = self.p_pad(pep)
        if ps: print("Padded", mhcp0.shape, tcr0.shape, pep0.shape)
        
        mhcp = self.m_max_1(mhcp0 + self.m_drop_1(self.m_ReLU_1(self.m_norm_1(self.m_conv_1(mhcp0)))))
        tcr = self.t_max_1(tcr0 + self.t_drop_1(self.t_ReLU_1(self.t_norm_1(self.t_conv_1(tcr0)))))
        #pep = self.p_max_1(self.p_drop_1(self.p_ReLU_1(self.p_norm_1(self.p_conv_1(pep0)))))
        if ps: print("Conv1", mhcp.shape, tcr.shape, pep.shape)

        mhcp = self.m_max_2(mhcp + self.m_drop_2(self.m_ReLU_2(self.m_norm_2(self.m_conv_2(mhcp)))))
        tcr = self.t_max_2(tcr + self.t_drop_2(self.t_ReLU_2(self.t_norm_2(self.t_conv_2(tcr)))))
        #pep = self.p_max_2(self.p_drop_2(self.p_ReLU_2(self.p_norm_2(self.p_conv_2(pep)))))
        if ps: print("Conv2", mhcp.shape, tcr.shape,pep.shape)

        mhcp = self.m_max_3(mhcp + self.m_drop_3(self.m_ReLU_3(self.m_norm_3(self.m_conv_3(mhcp)))))
        tcr = self.t_max_3(tcr + self.t_drop_3(self.t_ReLU_3(self.t_norm_3(self.t_conv_3(tcr)))))
        #pep = self.p_max_3(self.p_drop_3(self.p_ReLU_3(self.p_norm_3(self.p_conv_3(pep)))))
        if ps: print("Conv3", mhcp.shape, tcr.shape, pep.shape)

        mhcp = self.m_max_4(mhcp + self.m_drop_4(self.m_ReLU_4(self.m_norm_4(self.m_conv_4(mhcp)))))
        tcr = self.t_max_4(tcr + self.t_drop_4(self.t_ReLU_4(self.t_norm_4(self.t_conv_4(tcr)))))
        #pep = self.p_max_4(self.p_drop_4(self.p_ReLU_4(self.p_norm_4(self.p_conv_4(pep)))))
        if ps: print("Conv4", mhcp.shape, tcr.shape, pep.shape)

        mhcp = mhcp.view(bs0, 32*n_hid)
        tcr = tcr.view(bs0, 32*n_hid)
        #pep = pep.view(bs0, 640)
        if ps: print("Views", mhcp.shape, tcr.shape, pep.shape)

        mhcp = self.mrp_ReLU(self.mrp_norm(self.mrp_Linear(mhcp)))
        tcr = self.trp_ReLU(self.trp_norm(self.trp_Linear(tcr)))
        #pep = self.prp_ReLU(self.prp_norm(self.prp_Linear(pep)))
        if ps: print("Pre-rp", mhcp.shape, tcr.shape, pep.shape)
        
        #Random projection -> 640
        mhcp = mhcp * m1[0:bs0]
        tcr = tcr * m2[0:bs0]
        #pep = pep * m1[0:bs0]
        #if ps: print("RPed", mhcp.shape, tcr.shape, pep.shape)
        
        
        allparts = mhcp + tcr
        if ps: print("Allparts", allparts.shape)
        
        #Linear module
        allparts = self.a_linear(allparts)
        allparts = self.a_sigmoid(allparts)
        if ps: print("Last", allparts.shape)
        ps = False
        
        x = allparts
        #print(x[-1])
        return x

#print(net)  
#print( 'trainable params: {}'.format(count_model_params(net)) )
net = Model().cuda()

my_learner = Learner(my_data_bunch,
                     net,
                     opt_func=torch.optim.Adam,
                     loss_func=criterion, metrics=[accuracy],
                    wd = 0.01)
#opt_func=torch.optim.Adam
#my_learner.fit_one_cycle(1, 1e-02, wd = 0.01)


In [ ]:
my_learner.fit_one_cycle(4, max_lr=slice(None, 1e-02, 7e-02), wd = 0.01)
epochs += 1

In [ ]:
#Valid
Correct: 0.854
AUC 0.855
MCC 0.71
AVP 0.864

Tn, fp, fn, tp
[[191  21]
 [ 39 161]]
TPR: 0.805
TNR: 0.901
    
#test


### Resnet idea aminos only 085

In [ ]:
#Valid
Correct: 0.85
AUC 0.872
MCC 0.7
AVP 0.845

Tn, fp, fn, tp
[[176  36]
 [ 26 174]]
TPR: 0.87
TNR: 0.83
    
#Test
Correct: 0.824
AUC 0.853
MCC 0.648
AVP 0.731

Tn, fp, fn, tp
[[92 16]
 [21 81]]
TPR: 0.794
TNR: 0.852